In [4]:
import praw
import pandas as pd
from datetime import datetime

SECRET = "fDTjgQd2sS6gmVrqm7IcPPhp05NHXg"
APP_ID = "KGv2MMUcdlqidiWTIP4YPA"

reddit = praw.Reddit(
        client_id=APP_ID,
        client_secret=SECRET,
        user_agent="Comment Extraction")

search_query = "Elon Musk"
max_comments = 1000

comment_bodies = []
comment_authors = []
comment_upvotes = []
comment_awards = []
comment_dates = []

start_date = datetime(2023, 7, 24)
end_date = datetime(2023, 7, 30)

for submission in reddit.subreddit("all").search(search_query, limit=max_comments):
    submission.comments.replace_more(limit=0)
    for comment in submission.comments:
        if comment.is_root:
            comment_date = datetime.utcfromtimestamp(comment.created_utc)
            if start_date <= comment_date <= end_date:
                comment_bodies.append(comment.body)
                comment_authors.append(comment.author.name if comment.author else "[deleted]")
                comment_upvotes.append(comment.score)
                comment_awards.append(comment.total_awards_received)
                comment_dates.append(comment_date)

data = {
    "Comment_Body": comment_bodies,
    "Author": comment_authors,
    "Upvotes": comment_upvotes,
    "Awards": comment_awards,
    "Date": comment_dates,
}

df = pd.DataFrame(data)

df = df[df['Author'] != 'AutoModerator']
df = df[df['Comment_Body'] != '[deleted]']
df = df[~df['Comment_Body'].str.contains(r'\[Banned for overrating\]', case=False)]
df = df[~df['Comment_Body'].str.contains(r'\br/\w+', case=False)]

df

,Comment_Body,Author,Upvotes,Awards,Date
1,"“I liked the post so much I rexed, re-x’d, rex...",FungusFly,4003,0,2023-07-27 20:27:38
2,...so how much is the company worth now? About...,Lithaos111,3705,0,2023-07-27 21:23:58
3,As in my favorite ex-social media app.,HAMmerPower1,304,0,2023-07-27 21:06:23
4,So then it's official.\n\nIt's an X platform.\...,Jeoshua,808,0,2023-07-27 21:38:38
5,"#tweets are now called ""skeets""",Kbdiggity,735,0,2023-07-27 22:08:36
...,...,...,...,...,...
6047,"Elon Musk most likely has a disability, his mo...",Loneskumlord,1,0,2023-07-28 17:36:42
6048,Yes. He's a grade A narcissist.,Possible-Kangaroo635,1,0,2023-07-28 18:45:59
6049,He started to believe the PR stunt they made b...,Mantiax,1,0,2023-07-28 20:37:39
6050,Musk is a reincarnation of Howard Hughes. He...,TigerPoppy,1,0,2023-07-28 21:22:33


In [8]:
df = df.assign(Comment_Body_Length=df['Comment_Body'].str.len()).sort_values('Comment_Body_Length', ascending=False)
df = df.head(2000)
df = df.drop(columns='Comment_Body_Length')

df

,Comment_Body,Author,Upvotes,Awards,Date
1605,This is evident with the differences between h...,BadBoysWillBeSpanked,1,0,2023-07-28 16:20:38
2479,I have been warning about this for 2 years. Ho...,Hatrct,3,0,2023-07-26 00:57:32
769,"Tesla / Space X:\n\n* All the sexual assault, ...",Chroko,2,0,2023-07-28 07:43:22
3639,I firmly believe there is no endgame. This who...,Joec1211,10,0,2023-07-24 20:30:07
1452,Unfortunately unpopular opinion:\n\nElon musk ...,SkonerTM,0,0,2023-07-29 22:41:01
...,...,...,...,...,...
147,I just can't help but wonder if he is playing ...,XHandsomexJackx,2,0,2023-07-28 04:55:09
5643,That is such an unimaginative and petty list a...,Surur,-17,0,2023-07-28 10:34:27
2681,Good to see the hearings methods are working t...,dandilion788,1,0,2023-07-27 18:17:28
5917,x.com was a farce. Elon actually tried coding ...,oht7,1,0,2023-07-25 05:00:55


In [9]:
# Specify the file path where you want to save the CSV file
csv_file_path = "reddit_comments_ElonMusk.csv"

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)